In [ ]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import math
import glidertools as gt
import cmocean.cm as cmo
import matplotlib.pyplot as plt

from esdglider import gcp, glider, plots, utils

deployment_info = {
    "deployment_name": "amlr03-20231128",
    "mode": "delayed",
}
# deployment_info = {
#     "deployment_name": 'calanus-20241019', 
#     "mode": 'delayed', 
# }

# project = deployment_info["project"]
# year = utils.year_path(project, deployment)
bucket_name = 'amlr-gliders-deployments-dev'
deployments_path = f"/home/sam_woodman_noaa_gov/{bucket_name}"
config_path = f"/home/sam_woodman_noaa_gov/glider-lab/deployment-configs"

gcp.gcs_mount_bucket(
    "amlr-gliders-deployments-dev", deployments_path, 
    ro=False)

deployment_info["deploymentyaml"] = os.path.join(
    config_path, f"{deployment_info["deployment_name"]}.yml")

paths = glider.get_path_deployment(deployment_info, deployments_path)

In [ ]:
ds_eng = xr.load_dataset(paths["tsengpath"])
ds_sci = xr.load_dataset(paths["tsscipath"])
ds_g5 = xr.load_dataset(paths["gr5path"])

## Misc checks and work

In [ ]:
ds_raw = xr.load_dataset(paths["tsrawpath"])
x0 = ds_raw.sel(time=slice("2024-11-01 18:00", "2024-11-01 22:00"))

# from pyglider import utils as pgutils

# x2 = pgutils.get_glider_depth(x)
# x2
x0

In [ ]:
print(np.max(ds_sci.depth.values))
print(np.argmax(ds_sci.depth.values))
ds_sci.time[516252]

x = ds_sci.sel(time=slice("2024-11-01 18:00", "2024-11-01 22:00"))

# from pyglider import utils as pgutils

# x2 = pgutils.get_glider_depth(x)
# x2
x

In [ ]:
import matplotlib.pyplot as plt 

fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(ds_eng.gps_longitude, ds_eng.gps_latitude, s=3)

# # Labels and title
# ax.set_xlabel("Longitude", size=label_size)
# ax.set_ylabel("Latitude", size=label_size)
# ax.set_title(title_str, size=title_size)
ax.grid(True)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(ds_raw.gps_longitude, ds_raw.gps_latitude, s=3)

# # Labels and title
# ax.set_xlabel("Longitude", size=label_size)
# ax.set_ylabel("Latitude", size=label_size)
# ax.set_title(title_str, size=title_size)
ax.grid(True)
plt.show()

In [ ]:
y = ds_eng.sel(time=slice("2024-11-01 18:00", "2024-11-01 22:00"))
y

## Test esdglider.plots functions

In [ ]:
plots.sci_spatialgrid_plot("temperature", ds_g5, show=False)

In [ ]:
# Glidertools plotting
plots.sci_gt_plot(ds_sci, "density", show = False)

dat = ds_sci.where(ds_sci['profile_index'] % 1 == 0, drop = True)
x = dat.profile_index
y = dat.depth

fig, ax = plt.subplots(figsize=(11, 8.5))
ax = gt.plot(x, y, dat.density, cmap=cmo.dense, ax=ax, robust=True)\

gt.plot.bin_size(ds_sci.depth)

## Testing for making the surface map plots

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# from matplotlib_scalebar.scalebar import ScaleBar

bar_file = "/home/sam_woodman_noaa_gov/ETOPO_2022_v1_15s_N45W135_erddap.nc"
bar = xr.load_dataset(bar_file).rename({"latitude": "lat", "longitude": "lon"})
bar = bar.where(bar.z <= 0, drop=True)


plots.sci_surface_map(
    ds_g5, 
    "temperature", 
    crs=ccrs.Mercator(), 
    show=False, 
    bar=None, 
)

# # plt.show()

deployment = ds_g5.deployment_name
# start = ds.deployment_start[0:10]
# end = ds.deployment_end[0:10]

# map_lon_border = 0.1
# map_lat_border = 0.2
# glider_lon_min = ds.longitude.min()
# glider_lon_max = ds.longitude.max()
# glider_lat_min = ds.latitude.min()
# glider_lat_max = ds.latitude.max()
    
# # Cartopy
# fig, ax = plt.subplots(
#     figsize=(8.5, 11), subplot_kw={"projection": ccrs.Mercator()}
# )
# ax.set_xlabel("\n\n\nLongitude [Deg]", size=14)
# ax.set_ylabel("Latitude [Deg]\n\n\n", size=14)

# # Set extent of the map based on the glider data
# ax.set_extent(
#     [
#         glider_lon_min - map_lon_border,
#         glider_lon_max + 3 * map_lon_border,
#         glider_lat_min - map_lat_border,
#         glider_lat_max + map_lat_border,
#     ],
#     crs=ccrs.PlateCarree(),
# )

# # Add coastlines, countries, and continents
# # ax.coastlines(resolution="10m", color="black")
# # ax.add_feature(cfeature.BORDERS, linestyle="-", edgecolor="black")
# ax.add_feature(cfeature.LAND, edgecolor="black", facecolor="#e0b479")
# ax.add_feature(cfeature.OCEAN, edgecolor="none", facecolor="#7bcbe3")

# gl = ax.gridlines(draw_labels=["bottom", "left"])
# gl.xlabel_style = {'rotation': 15}

# # for label in ax.get_xticklabels():
# #     label.set(rotation=-30)

# # plt.show()

# # ax.add_artist(ScaleBar(1, location="lower right"))


# plots.sci_timeseries_loop(ds_sci, None, show=True)
plots.eng_timeseries_loop(ds_eng, None, show=True)